In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

In [5]:
# import spin-up files
ds = xr.open_dataset('/burg/abernathey/users/hillary/spin_up/QG_zhang20_config_1041600.nc')
ds

<xarray.Dataset>
Dimensions:  (k: 257, l: 512, lev: 2, time: 1, x: 512, y: 512)
Coordinates:
  * k        (k) float64 0.0 5.236e-06 1.047e-05 ... 0.00133 0.001335 0.00134
  * l        (l) float64 0.0 5.236e-06 1.047e-05 ... -1.047e-05 -5.236e-06
  * lev      (lev) int64 1 2
  * time     (time) timedelta64[ns] 12 days 01:20:00
  * x        (x) float64 1.172e+03 3.516e+03 5.859e+03 ... 1.196e+06 1.199e+06
  * y        (y) float64 1.172e+03 3.516e+03 5.859e+03 ... 1.196e+06 1.199e+06
Data variables:
    Qy       (lev) float64 ...
    Ubg      (lev) float64 ...
    q        (time, lev, y, x) float64 ...
    u        (time, lev, y, x) float64 ...
    ufull    (time, lev, y, x) float64 ...
    v        (time, lev, y, x) float64 ...
    vfull    (time, lev, y, x) float64 ...
Attributes: (12/18)
    L:          1200000.0
    W:          1200000.0
    dt:         600.0
    filterfac:  23.6
    nk:         257
    nl:         512
    ...         ...
    tavestart:  311040000
    tc:         1736
    tmax:       311040000
    twrite:     50000
    title:      pyqg: Python Quasigeostrophic Model
    reference:  https://pyqg.readthedocs.io/en/latest/index.html

In [8]:
ds

<xarray.Dataset>
Dimensions:  (k: 257, l: 512, lev: 2, time: 1, x: 512, y: 512)
Coordinates:
  * k        (k) float64 0.0 5.236e-06 1.047e-05 ... 0.00133 0.001335 0.00134
  * l        (l) float64 0.0 5.236e-06 1.047e-05 ... -1.047e-05 -5.236e-06
  * lev      (lev) int64 1 2
  * time     (time) timedelta64[ns] 12 days 01:20:00
  * x        (x) float64 1.172e+03 3.516e+03 5.859e+03 ... 1.196e+06 1.199e+06
  * y        (y) float64 1.172e+03 3.516e+03 5.859e+03 ... 1.196e+06 1.199e+06
Data variables:
    Qy       (lev) float64 1.552e-10 -2.256e-11
    Ubg      (lev) float64 0.04 0.0
    q        (time, lev, y, x) float64 ...
    u        (time, lev, y, x) float64 ...
    ufull    (time, lev, y, x) float64 ...
    v        (time, lev, y, x) float64 ...
    vfull    (time, lev, y, x) float64 ...
Attributes: (12/18)
    L:          1200000.0
    W:          1200000.0
    dt:         600.0
    filterfac:  23.6
    nk:         257
    nl:         512
    ...         ...
    tavestart:  311040000
    tc:         1736
    tmax:       311040000
    twrite:     50000
    title:      pyqg: Python Quasigeostrophic Model
    reference:  https://pyqg.readthedocs.io/en/latest/index.html

In [ ]:
# Mean Eddy Kinetic Energy (EKE) as a function of time
dEKE_dt = np.gradient(ds.EKE.isel(lev=0), ds.time)
I = int(np.where(np.abs(ds.time - year*14)==np.min(np.abs(ds.time - year*14)))[0])
time_stable = ds.time[I].values

# Plot EKE derivative
plt.rcParams.update({'font.size': 18})
plt.figure(figsize=(12,6)); plt.plot(ds.time/year, dEKE_dt)
plt.ylabel('EKE central differece'); plt.xlabel('model time (year)')
plt.axvline(x=time_stable/year, color='k', linestyle='--')


print('model appears stable at', ds.time[I].values/year, 'years');

In [ ]:
# Plot time series of EKE
plt.figure(figsize=(12,6))
plt.semilogy(ds.time/year, ds.EKE.isel(lev=0), lw=4, color='purple', label='upper layer')
plt.semilogy(ds.time/year, ds.EKE.isel(lev=1), lw=4, color='darkorange', label='lower layer')
ints = [10000, 15000, 25000, 35000, 55000, 70000]
for i in ints:
    plt.axvline(x=ds.time[i]/year, color='lightblue', linewidth=2, alpha=1)
plt.axvline(x=time_stable/year, color='k', lw=2, linestyle='--')
plt.grid(True, alpha=0.5); plt.legend(frameon=False, ncol=2)
plt.ylabel(r'KE ($\rmcm^{2}$/$\rms^{2}$)'); plt.xlabel('Model Time (year)');

As the model gets spun up, the mean EKE increases until it reaches an equilibrated state. When the EKE plateaus the model is in a stable state. The dashed vertical line marks 12.5 years when the model appears stable. This would be a good place to save the model state for the ensemble perturbation experiment.

Vertical light blue lines show times corresponding the the snapshots plotted below. 

In [ ]:
# Evolution of the upper PV anomaly field
print('stable index ', int(np.where(ds.time == time_stable)[0]))

plt.rcParams.update({'font.size': 10})
plt.rcParams['image.cmap'] = 'RdBu'

ints = [0, 1000, 5000, 10000, 20000, 50000]
plt.figure(figsize=(15,15))
for i in enumerate(ints):
    plt.subplot(2,3,i[0]+1)
    plt.pcolormesh(ds.q[i[1],0,:,:])
    plt.title((ds.time[i[1]].values/year).round(decimals=2), color='k') # The titles correspond to the model time in years. 
    plt.colorbar()

In [ ]:
# Save model state at equilibrium
I = int(np.where(np.abs(ds.time - time_stable)==np.min(np.abs(ds.time - time_stable)))[0])
qg_equilibrium = ds[dict(time=I)]

path = '/burg/abernathey/users/hillary/'
qg_equilibrium.to_netcdf(path+'QG_equilibrium_proto.nc', engine='h5netcdf', invalid_netcdf=True)